In [43]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings()

In [44]:
http = urllib3.PoolManager()

url = 'http://www.ou.edu/content/web/resources_offices/content/academic_departments.html'

req = http.request('GET', url)
print(req.status)

from bs4 import BeautifulSoup
soup = BeautifulSoup(req.data, 'html.parser')
# create groupings based ong the class = 'parbae section text'
div_list = soup.findAll('div',{'class':'parbase section text'})


# now loop through the groupings looking for:
#    <h3> --> campus
#    <a > --> college
#    <li> --> department

departments =[]
college_dict = {}

for item in div_list:
   
    if item.find('h3'):
        print(item.find('h3').getText().title())
#        print('++++++++++++++++++')


    if item.find('a'):
        print(item.find('a').getText().replace('\n','').title())
        college = item.find('a').getText().replace('\n','').title()
        college_dict.setdefault(college, [])
        dept_lists = item.findAll('li')
        for item2 in dept_lists:
            department = item2.getText().replace('\n','')
#            print ('\t', department)
            college_dict[college].append(department)
            departments.append(item2.getText().replace('\n',''))
        print('------------------')
    else:
        next


200
Norman Campus
Office Of The Senior Vice  President And Provost
------------------
University College
------------------
College Of Architecture
------------------
College Of Arts And Sciences
------------------
College Of Atmospheric & Geographic Sciences
------------------
Michael F. Price College Of Business
------------------
Mewbourne College Of Earth And Energy
------------------
Jeannine Rainbolt College Of Education
------------------
Gallogly College Of Engineering
------------------
Weitzenhoffer Family College Of Fine Arts
------------------
Graduate College
------------------
Gaylord College Of Journalism And Mass Communication
------------------
College Of Law
------------------
Honors College
------------------
Reserve Officers Training Corps
------------------
College Of International Studies
------------------
Ou Extended Campus
------------------
Ou Outreach - College Of Continuing Education
------------------
Health Sciences Center
Office Of The Senior Vice  Presid

Fixes for departments in dictionary either missrepresetned or missing

In [45]:
fixes = []
fixes = [('College Of Atmospheric & Geographic Sciences','Department of Geography and Environmental Sustainability'),
         ('Michael F. Price College Of Business', 'MIS'),
         ('College Of Arts And Sciences', 'French'),
         ('OU Libraries', 'OU Libraries'), 
         ('OUIT', 'OSCER'),
         ('OUIT', 'Information Technology'),
         ('Centers', 'Center'),
         ('Michael F. Price College Of Business', 'College of Business'),
         ('Gallogly College Of Engineering', 'College Of Engineering')
        ]
for college, department in fixes:

    if college not in college_dict.keys():
        college_dict.setdefault(college, [])

    college_dict[college].append(department)
    departments.append(department)


In [48]:
ws_department = pd.read_csv('data/departments_served-new.csv')

In [49]:
ws_department

,department
0,Microbiology and Plant Biology
1,Biology
2,Biology
3,Microbiology and Plant Biology
4,Microbiology and Plant Biology
5,Geography and Environmental Sustainability
6,Biology
7,Microbiology and Plant Biology
8,MIS
9,History


In [50]:
from fuzzywuzzy import fuzz, process

count = 1
for dept in ws_department.loc[:]['department']:
    print(count, dept)
    result = process.extract(dept, departments, limit=1,scorer=fuzz.token_set_ratio)
    print(result,'\n')
    count += 1

In [51]:
def find_key(dictionary, value):
    for key, values in dictionary.items():
            if value in values:
                return(key)


Setting for a specific scorer.

In [52]:
count = 1
college_list = []
dept_nohome = []

for dept in ws_department.loc[:]['department']:
#    print(count, dept)
    result = process.extract(dept, departments, limit=1, scorer=fuzz.token_set_ratio)
#    print(result,'\n')
    count += 1
    if result[0][1] > 95:
        college_key = find_key(college_dict,result[0][0])
        college_list.append(college_key)
    else:
#        print('No College found \n', result, '\n')
        dept_nohome.append(dept) 
        
        
#    print('\n')

In [53]:
counts = {}
for item in set(college_list) :
    counts.setdefault(item, college_list.count(item))

sorted_d = sorted((value, key)for (key,value) in counts.items())

final_sort = sorted(sorted_d, reverse=True)
final_sort

[(86, 'College Of Arts And Sciences'),
 (37, 'College Of Atmospheric & Geographic Sciences'),
 (24, 'OU Libraries'),
 (14, 'Gallogly College Of Engineering'),
 (13, 'Mewbourne College Of Earth And Energy'),
 (8, 'Centers'),
 (5, 'OUIT'),
 (3, 'Ou Outreach - College Of Continuing Education'),
 (3, 'Michael F. Price College Of Business'),
 (3, 'College Of Architecture'),
 (1, 'Weitzenhoffer Family College Of Fine Arts'),
 (1, 'Ou Extended Campus'),
 (1, 'College Of Pharmacy')]

In [54]:
counts

{'College Of Pharmacy': 1,
 'College Of Arts And Sciences': 86,
 'College Of Atmospheric & Geographic Sciences': 37,
 'College Of Architecture': 3,
 'OU Libraries': 24,
 'OUIT': 5,
 'Mewbourne College Of Earth And Energy': 13,
 'Weitzenhoffer Family College Of Fine Arts': 1,
 'Centers': 8,
 'Ou Outreach - College Of Continuing Education': 3,
 'Ou Extended Campus': 1,
 'Gallogly College Of Engineering': 14,
 'Michael F. Price College Of Business': 3}

In [55]:
df = pd.DataFrame(final_sort)
df

,0,1
0,86,College Of Arts And Sciences
1,37,College Of Atmospheric & Geographic Sciences
2,24,OU Libraries
3,14,Gallogly College Of Engineering
4,13,Mewbourne College Of Earth And Energy
5,8,Centers
6,5,OUIT
7,3,Ou Outreach - College Of Continuing Education
8,3,Michael F. Price College Of Business
9,3,College Of Architecture


In [56]:
print(dept_nohome)

['CIMMS', 'OMRF', 'Oklahoma Geological Survey', 'Information and Journalism', 'Molecular Anthropology', 'SWOSU/Computer Science', 'SWOSU/Computer Science', 'USDA - ARS', 'USDA - ARS', 'Chickasaw Nation IT', 'USDA - ARS', 'Digital Metadata Services', 'HSCI', 'Digital Metadata Services', 'HSCI', 'Oklahoma Geological Survey', 'Pediatrics', 'College of Atmosphere and Geographical Sciences', 'Oklahoma Geological Survey', 'Oklahoma Geological Survey', 'Oklahoma Geological Survey', 'American Organ Institute', 'National Weather Service', 'University College', 'MPGE', 'Institute for Environmental Genomics', 'Schusterman Library OU-Tulsa', 'Institute for Environmental Genomics', 'OU-Institute for Environmental Genomics', 'OU-Institute for Environmental Genomics', 'Institute for Environmental Genomics', 'UT Tyler', 'Institute for Environmental Genomics', 'University of Texas at Tyler', 'EOMF', 'EOMF', 'EOMF', 'AME', 'OBS', 'OU-Tulsa Research', 'OU-college of international studies', 'Ecology and E

In [57]:
for item in college_dict.keys():
    print(item)

Office Of The Senior Vice  President And Provost
University College
College Of Architecture
College Of Arts And Sciences
College Of Atmospheric & Geographic Sciences
Michael F. Price College Of Business
Mewbourne College Of Earth And Energy
Jeannine Rainbolt College Of Education
Gallogly College Of Engineering
Weitzenhoffer Family College Of Fine Arts
Graduate College
Gaylord College Of Journalism And Mass Communication
College Of Law
Honors College
Reserve Officers Training Corps
College Of International Studies
Ou Extended Campus
Ou Outreach - College Of Continuing Education
College Of Allied Health
College Of Dentistry
College Of Medicine
Fran And Earl Ziegler College Of Nursing
College Of Pharmacy
College Of Public Health
OU Libraries
OUIT
Centers


In [58]:
find_key(college_dict,'Geography')

In [59]:
college_dict.get('Ou Extended Campus')

['Advanced Placement - Summer Institute for Teachers',
 'Aviation',
 'Blowout Prevention School',
 'Center for Chamber of Commerce Excellence',
 'Center for Independent and Distance Learning',
 'Civic Orchestra',
 'Corrosion Control Course',
 'Developmental Studies',
 'Duplicating and Postal Services',
 'Economic Development Institute (OU/EDI)',
 'English as a Second Language, Center for',
 'Engineering/Geosciences Programs',
 'Feaver-MacMinn Seminar',
 'Forum and Conference Services',
 'Gas Compressor Short Course',
 'Insight Magazine',
 'Journal of Museum Studies',
 'Laurence Reid Gas Conditioning Conference',
 'Lean Institute',
 'Legal Issues in Higher Education Conference',
 'LSAT Prep',
 'MCAT Prep',
 'NCPACE',
 'Osher Lifelong Learning Institute',
 'OU North America and Europe (Graduate Degrees)',
 'OU Online (Undergraduate and Graduate Degrees)',
 'Precollegiate Programs',
 'Project Managment',
 'Reading: Speed Reading and Study Strategies',
 'Reading: Summer Reading Skills for 

In [60]:
college_dict

{'Office Of The Senior Vice  President And Provost': [],
 'University College': [],
 'College Of Architecture': ['Division of Architecture',
  'Division of Construction Science',
  'Division of Interior Design',
  'Division of Landscape Architecture',
  'Division of Regional and City Planning'],
 'College Of Arts And Sciences': ['Advanced Center for Genome Technology',
  'African and African-American Studies Program',
  'Department of Anthropology',
  'Center of Applied Research for Non-Profit Organizations',
  'Department of Biology',
  'Department of Chemistry and Biochemistry',
  'Department of Classics & Letters',
  'Department of Communication',
  'Confucius Institute',
  'Department of Economics',
  'Department of English',
  'Film and Media Studies Program',
  'Department of Health and Exercise Science',
  'Department of History',
  'Department of History of Science',
  'Department of Human Relations',
  'Environmental Studies',
  'Interdisciplinary International Relations Progr